Toteat explorer data

In [2]:
import requests

response = requests.get('https://storage.googleapis.com/backupdatadev/ejercicio/ventas.json')
 
# print response
print(response)
data = response.json()

<Response [200]>


In [3]:
data[0]

{'date_closed': '2019-03-18 15:40:47',
 'zone': 'Salón',
 'waiter': 'María José Perez',
 'cashier': 'Roberto Ortega',
 'products': [{'category': 'Bebidas',
   'price': 2000,
   'name': 'Café',
   'quantity': 1},
  {'category': 'Tragos', 'price': 2500, 'name': 'Cerveza', 'quantity': 1}],
 'diners': 2,
 'date_opened': '2019-03-18 14:04:01',
 'table': 7,
 'total': 4500,
 'id': '1ea97de85eb634d580161c603422437f',
 'payments': [{'amount': 4500, 'type': 'Efectivo'}]}

# Diferencias de precio mismo producto

In [4]:
vals = set()
from collections import defaultdict 

for order in data:
  actual_price = 0
  products = {}
  for prod in order["products"]:
    actual_price += prod["price"]*prod["quantity"]
    if prod["name"] in products and products[prod["name"]] != prod["price"]:
      print("Problema")
      print("name", prod["name"])
    else:
      products[prod["name"]] = prod["price"]
products
# No hay


{'Ensalada': 6250,
 'Coca Cola': 1500,
 'Bisteck con acompañamiento': 10000,
 'Limonada': 1500,
 'Pollo asado': 9000,
 'Café': 2000,
 'Cerveza': 2500,
 'Ración chorizos': 4000}

# Diferencias en suma de precios y total

Encontramos varias diferencias, estas probablemente estan relacionadas con las propinas que reciben los garzones. Aun así dadas las preocupaciones de Antonio, puede ser bueno tener una mejor visualización de estos casos.

In [5]:
vals = set()

for order in data:
  actual_price = 0
  payments_price = 0
  for prod in order["products"]:
    actual_price += prod["price"]*prod["quantity"]
  for payment in order["payments"]:
    payments_price += payment["amount"]
  if payments_price != order["total"]:
    print( order["waiter"], order["cashier"])
    print(payments_price)
    print("diff payment and price: ", 100*(payments_price - order["total"])/payments_price, "%")

  if actual_price != order["total"]:
    print( order["waiter"], order["cashier"])
    print(actual_price)
    print("diff: ", 100*(actual_price - order["total"])/actual_price, "%")

María José Perez Sebastian Hernandez
1500
diff:  6.0 %
Jose Gonzalez Michael Orletto
73250
diff:  0.9556313993174061 %
Pedro Vargas Michael Orletto
95750
diff:  0.10966057441253264 %
Katherine Yañez Roberto Ortega
92250
diff:  0.2926829268292683 %
Diana Arevalo Michael Orletto
37000
diff:  1.2162162162162162 %
María José Perez Michael Orletto
52250
diff:  0.14354066985645933 %
Diana Arevalo Roberto Ortega
122000
diff:  0.29508196721311475 %
Pedro Vargas Michael Orletto
103500
diff:  0.8695652173913043 %
Rodrigo Matamala Sebastian Hernandez
104500
diff:  0.17894736842105263 %
Jesus Romero Michael Orletto
106750
diff:  0.48711943793911006 %
Jose Gonzalez Michael Orletto
107000
diff:  1.2616822429906542 %
María José Perez Sebastian Hernandez
64750
diff:  0.23166023166023167 %
Jose Gonzalez Roberto Ortega
50500
diff:  0.49504950495049505 %
Jose Gonzalez Sebastian Hernandez
81500
diff:  0.5337423312883436 %
Diana Arevalo Michael Orletto
92750
diff:  0.9703504043126685 %
Jesus Romero Roberto

# Date info


In [19]:
from datetime import datetime

In [20]:
def parse_time(date: str):
  return datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

In [33]:
dates_utils = {
    "first_open": parse_time(data[0]["date_opened"]),
    "last_closed": parse_time(data[0]["date_closed"]),
    "max_range": parse_time(data[0]["date_closed"]) 
      - parse_time(data[0]["date_opened"]),
    "close_date_max_range": parse_time(data[0]["date_closed"]),
}
mean_waiting_time = parse_time(data[0]["date_closed"]) - parse_time(data[0]["date_opened"])
for order in data:
  date_closed = parse_time(order["date_closed"])
  date_open = parse_time(order["date_opened"])
  mean_waiting_time += date_closed - date_open
  
  if date_open < dates_utils["first_open"]:
    dates_utils["first_open"] = date_open
  
  if date_closed > dates_utils["last_closed"]:
    dates_utils["last_closed"] = date_closed

  if date_closed - date_open > dates_utils["max_range"]:
    dates_utils["max_range"] = date_closed - date_open
    dates_utils["close_date_max_range"] = date_closed

print(f"first opened order: ", dates_utils["first_open"])
print(f"last closed order: ", dates_utils["last_closed"])
print(f"bigger time order range: ", dates_utils["max_range"])
print(f"biggest range closed time: ", dates_utils["close_date_max_range"])

print(f"mean time order range: ", mean_waiting_time/len(data))
    



first opened order:  2019-01-01 11:01:25
last closed order:  2019-03-31 21:49:10
bigger time order range:  2:10:00
biggest range closed time:  2019-02-10 18:45:28
mean time order range:  1:12:16.668807


Nada muy extraño que salga a la vista